## Pacejka Formula


The aim of this project is to use the magic formula to fit curves to tire performance based on the emperical data given. We'll use these curves to determine the best tire to use during competition. 

In [52]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

### Magic Longitudinal Formula
This is a mathematical model used to calculate the longitudinal force of a tire based on its vertical load and longitudinal slip.

In [53]:
# SR = slip ratio
# B = stiffness factor
# C = shape factor
# D = peak factor
# E = curvature factor
def longitudinal_formula(SR, B, C, D, E):
    return D * np.sin(C * np.arctan(B * (1 - E) * SR + E * np.arctan(B * SR)))

### Magic Lateral Force Formula
This model describes the relationship between the lateral force and slip angle.

In [54]:
# SA = slip angle
# B = stiffness factor
# C = shape factor
# D = peak factor
# E = curvature factor
def lateral_formula(SA, B, C, D, E):
    SA_rad = np.radians(SA)
    return D * np.sin(C * np.arctan(B * (1 - E) * SA_rad + E * np.arctan(B * SA_rad)))

### Data Processing
We will now process the data from the .dat files provided by FSAE.

In [55]:
# Load in the data
csv_path = r"/Users/EmmyChen/IdeaProjects/pacejka-tire-modeling/research/B1965run32.dat"

data = pd.read_csv(csv_path, sep='\t', skiprows=2)

In [ ]:
data.head()

In [ ]:
# Extract relevant columns
SR = data['SR'].values
SA = data['SA'].values
FX = data['FX'].values
SA = data['SA'].values
FY = data['FY'].values
FZ = data['FZ'].values
data['SR'].describe()

In [ ]:
FX_df = pd.DataFrame(FX)
FX_df.dtypes
SR_df = pd.DataFrame(SR)
print(SR_df.isnull().any())
print(FX_df.isnull().any())
print(len(SR), len(FX))

In [ ]:
plt.scatter(SR, FX)
plt.show()

In [ ]:

# Fit the model
popt_fx, _ = curve_fit(longitudinal_formula, SR, FX, p0=[10, 1.5, max(FX), 0.5], maxfev=10000)

# Plot results
SR_fit = np.linspace(min(SR), max(SR), 200)
FX_fit = longitudinal_formula(SR_fit, *popt_fx)

plt.scatter(SR, FX, s=8, label="Data")
plt.plot(SR_fit, FX_fit, color='r', label="Magic Formula Fit")
plt.xlabel("Slip Ratio SR")
plt.ylabel("Fx (N)")
plt.legend()
plt.show()

print("Longitudinal parameters (B, C, D, E):", popt_fx)

In [ ]:
plt.scatter(SA, FY)
plt.show()

In [ ]:
popt_fy, _ = curve_fit(lateral_formula, SA, FY, p0=[10, 1.3, max(FY), 0.5], maxfev=10000)

SA_fit = np.linspace(min(SA), max(SA), 79843)
FY_fit = lateral_formula(SA_fit, *popt_fy)

plt.scatter(SA, FY, s=8, label="Data")
plt.plot(SA_fit, FY_fit, color='r', label="Magic Formula Fit")
plt.xlabel("Slip Angle (°)")
plt.ylabel("Fy (N)")
plt.legend()
plt.show()

print("Lateral parameters (B, C, D, E):", popt_fy)